In [31]:
import os
import random
import numpy as np
import librosa.display
import soundfile as sf
import json as json
import pandas as pd

In [2]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [52]:
'''
PATH INITIALIZATION

PATH = Origninal Audio Files
SAVE_PATH = Transformed Audio Files
META_DATA_PATH = Metadata.json location
'''
PATH = "/Users/valleotb/Desktop/Valleotb/sample"
SAVE_PATH = "/Users/valleotb/Desktop/Valleotb/sample_save"
META_DATA_PATH = '/Users/valleotb/Desktop/Valleotb/sample_metadata'
FILE_COUNT = get_files_count(PATH)

'''
NAMING SETTING
'''
j = 100000  # 원본 파일 네이밍
z = 200000  # 수정 후 파일 네이밍

'''
PARAMETER SETTING
'''
SAMPLE_TIME = 16000 * 3 # (16000 * second)
OUTPUT_DIMENSION = 10

'''
META DATA INITIALIZATION
'''
META_DATA = f"{META_DATA_PATH}/metadata.json"
DATA = {}
# DATA['speech_segments'] = []

for i in range(FILE_COUNT - 1):
    j += 1

    '''
    AUDIO PREPROCESSING (CUT TO SAME TIMES)
    '''
    WAVEFORM, SR = sf.read(f"{PATH}/{j}.flac")
    WAVEFORM = librosa.util.normalize(WAVEFORM)
    NON_SILENT_TIME = librosa.effects.split(WAVEFORM, top_db=20)

    WAVEFORM = WAVEFORM[NON_SILENT_TIME[0][0] : (NON_SILENT_TIME[0][0]+SAMPLE_TIME)]
    NON_SILENT_TIME_2 = librosa.effects.split(WAVEFORM, top_db=15)
    print(NON_SILENT_TIME_2)

    if len(NON_SILENT_TIME_2) > 2 :
        random_num = random.randint(0, len(NON_SILENT_TIME_2)-1)
        k = NON_SILENT_TIME_2[random_num]
        if (k[1] - k[0]) > (SAMPLE_TIME / OUTPUT_DIMENSION) :

            '''
            DATA TRANSFORM PROCESS
            '''
            # Pitch Shift Process
            N_STEPS = random.randint(1,5)
            WAVEFORM_TRANSFORM = librosa.effects.pitch_shift(WAVEFORM[k[0]:k[1]], sr=SR, n_steps=N_STEPS)
            WAVEFORM_TRANSFORM_MINUS = librosa.effects.pitch_shift(WAVEFORM_TRANSFORM, sr=SR, n_steps=-N_STEPS)

            # Normalize
            WAVEFORM_NOT_TRANSFORM_PART = WAVEFORM[k[0]:k[1]]
            WAVEFORM_DOUBLED = WAVEFORM_NOT_TRANSFORM_PART ** 2
            WAVEFORM_TRANSFORM_DOUBLED = WAVEFORM_TRANSFORM_MINUS ** 2
            a = np.sqrt((WAVEFORM_DOUBLED.sum()) / (WAVEFORM_TRANSFORM_DOUBLED.sum()))

            WAVEFORM_TRANSFORM_FINAL = WAVEFORM_TRANSFORM_MINUS * a
            WAVEFORM[k[0]:k[1]] = WAVEFORM_TRANSFORM_FINAL

            '''
            DATA LABELING PROCESS

            LABELING => 구간의 50% 이상이 포함되었을때만 Labeling 진행한다.
            LABEL_DATA, POINTER -> Initializing
            '''
            LABEL_DATA = []
            POINTER = 0

            sf.write(f'{SAVE_PATH}/{z}.wav', WAVEFORM, SR, subtype="PCM_24")

            for e in range(OUTPUT_DIMENSION):
                if POINTER + (SAMPLE_TIME / OUTPUT_DIMENSION) <= k[0]:
                    # save label
                    LABEL_DATA.append(0)
                    POINTER += (SAMPLE_TIME / OUTPUT_DIMENSION)     # POINTER += 4800

                elif k[0] < POINTER + (SAMPLE_TIME / OUTPUT_DIMENSION) <= k[1]:
                    # save label
                    LABEL_DATA.append(1)
                    POINTER += (SAMPLE_TIME / OUTPUT_DIMENSION)     # POINTER += 4800

                elif POINTER + (SAMPLE_TIME / OUTPUT_DIMENSION) > k[1]:
                    # save label
                    LABEL_DATA.append(0)
                    POINTER += (SAMPLE_TIME / OUTPUT_DIMENSION)     # POINTER += 4800

            # write metadata
            DATA[f"{z}.wav"] = LABEL_DATA

            # process end
            print(f'complete{z}')
            z += 1

with open(META_DATA, 'w') as outfile:
    json.dump(DATA, outfile)

[[    0 27648]
 [28160 48000]]
[[  512 12800]
 [16896 19456]
 [22528 26112]
 [34816 44544]
 [46592 48000]]
complete200000
[[    0   512]
 [ 1024 17920]
 [23552 48000]]
[[    0 25600]
 [26112 29184]
 [37888 40960]
 [41472 46592]]
complete200001
[[    0  5632]
 [ 6144 13824]
 [19456 34304]
 [35328 45056]
 [46592 48000]]
complete200002
[[    0  9216]
 [ 9728 12800]
 [13824 17920]
 [20480 22528]
 [23552 32256]
 [34816 39424]
 [42496 48000]]
[[    0 30208]
 [30720 48000]]
[[    0  5632]
 [ 6144 17408]
 [21504 30720]
 [34304 48000]]
complete200003
[[  512 31232]
 [37376 44032]
 [47104 48000]]
complete200004
[[    0 17408]
 [18432 24576]
 [25600 31232]
 [33280 40448]
 [41984 47104]]
complete200005
[[    0 13824]
 [15360 18432]
 [22528 28160]
 [30208 33280]
 [35840 37376]
 [45056 47104]]
complete200006
[[  512  7168]
 [ 8192 17920]
 [19968 23552]
 [26112 31744]
 [33792 38912]
 [40448 46592]]
complete200007
[[  512 19968]
 [22528 26624]
 [27136 29696]
 [30208 37888]
 [40960 48000]]
complete2000

In [53]:
# meta_data_json = pd.read_json('/Users/valleotb/Desktop/Valleotb/sample_metadata/metadata.json')
meta_data_json = json.load(open('/Users/valleotb/Desktop/Valleotb/sample_metadata/metadata.json'))
print(meta_data_json)

{'200000.wav': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], '200001.wav': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], '200002.wav': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], '200003.wav': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1], '200004.wav': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0], '200005.wav': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0], '200006.wav': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], '200007.wav': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], '200008.wav': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], '200009.wav': [0, 0, 0, 0, 0, 1, 1, 1, 0, 0], '200010.wav': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1], '200011.wav': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], '200012.wav': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0], '200013.wav': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0]}


In [56]:
m = list(meta_data_json.keys())
m[0]

'200000.wav'

In [62]:
meta_data_json[m[0]]

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [63]:
print(m)

['200000.wav', '200001.wav', '200002.wav', '200003.wav', '200004.wav', '200005.wav', '200006.wav', '200007.wav', '200008.wav', '200009.wav', '200010.wav', '200011.wav', '200012.wav', '200013.wav']


In [64]:
print(len(meta_data_json))

14
